In [19]:
import os

import numpy as np
import pandas as pd

os.environ [ "KMP_DUPLICATE_LIB_OK" ]= "TRUE"

%load_ext autoreload
%matplotlib notebook
%autoreload 2
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Preparation

In [20]:
from common import *
from utils import *
from mydataset import *
import mydataset
reload(loss)
reload(plot)
reload(mydataset)
from my_collate_fn import my_collate_fn_3
from loss import validate_KL
from Config.config_MDN import DefaultConfig

In [21]:
# MODEL_NAME = "GT1_GT2_EMD"
#
# INPUT_LIST = get_Input_List(MODEL_NAME)

In [22]:
# opt = DefaultConfig(MODEL_NAME=MODEL_NAME)
# # seed = 223
#
# q=opt.q
# lr_for_pi = opt.lr_for_pi
# lr_for_shape = opt.lr_for_shape
# lr_for_scale = opt.lr_for_scale
# learning_rate = opt.learning_rate
#
# MB_MDN_Flag = opt.Conv_1_4
#
# batch_size = opt.batch_size
# EPOCH_NUM = opt.EPOCH_NUM
#
# opt.ARTIFICIAL, MODEL_NAME

In [23]:
def get_params(mlp):
    """
    Set learning rates for different layers and return params for training
    Args:
        mlp:
        opt:

    Returns: params

    """
    shape_params = list(map(id, mlp.z_shape.parameters()))
    scale_params = list(map(id, mlp.z_scale.parameters()))
    pi_params = list(map(id, mlp.z_pi.parameters()))

    params_id = shape_params + scale_params + pi_params

    base_params = filter(lambda p: id(p) not in params_id, mlp.parameters())
    params = [{'params': base_params},  # 如果对某个参数不指定学习率，就使用最外层的默认学习率
            {'params': mlp.z_pi.parameters(), 'lr': lr_for_pi},
            {'params': mlp.z_shape.parameters(), 'lr': lr_for_shape},
            {'params': mlp.z_scale.parameters(), 'lr': lr_for_scale}]

    return params

# 3. Test
## 3.1 Draw

In [24]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## 3.2 Data

In [25]:
# dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.data_key_path, opt.NLL_metric_path)
#
# shuffled_indices = save_data_idx(dataset, opt.arr_flag)
# train_idx, val_idx, test_idx = get_data_idx(shuffled_indices,opt.train_pct, opt.vali_pct,opt.SET_VAL)

## 3.3 Test: all data set

In [27]:
THRESHOLD = 0 # 现在是小于Th的被考虑
MODEL_LIST = ["GT2(MDN)","GT1+EMD", "GT2+EMD", "GT1+GT2", "GT1_GT2_EMD"] #
# MODEL_LIST = ["GT1(MDN)", "GT2(MDN)","EMD", "GT1+EMD", "GT2+EMD", "GT1+GT2", "GT1_GT2_EMD"] #


In [28]:
running_times = 3
metric_pd = pd.DataFrame()

for MODEL_NAME in MODEL_LIST:
    INPUT_LIST = get_Input_List(MODEL_NAME)

    print(f"========== MODEL_NAME = {MODEL_NAME} | INPUT_LIST = {INPUT_LIST} ==========")
    opt = DefaultConfig(MODEL_NAME=MODEL_NAME)

    q=opt.q
    lr_for_pi = opt.lr_for_pi
    lr_for_shape = opt.lr_for_shape
    lr_for_scale = opt.lr_for_scale
    learning_rate = opt.learning_rate

    MB_MDN_Flag = opt.Conv_1_4

    batch_size = opt.batch_size
    EPOCH_NUM = opt.EPOCH_NUM

    dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.data_key_path)

    my_collate_fn = functools.partial(my_collate_fn_3, INPUT_LIST=INPUT_LIST)

    # for seed in seed_list:
    setup_seed(opt.seed)

    print(f"========== opt.seed = {opt.seed} ==========")

    shuffled_indices = save_data_idx(dataset, opt.arr_flag)
    train_idx, val_idx, test_idx = get_data_idx(shuffled_indices,opt.train_pct, opt.vali_pct)

    _,_,test_loader = get_data_loader(dataset, opt.batch_size, train_idx, val_idx, test_idx, my_collate_fn)

    ans = []
    pred_metric = []

    for i in range(running_times):
        model = Conv_1_4(opt.N_gaussians, ch_in=len(INPUT_LIST)).to(device)
        model_path = get_MDN_save_path(opt.ARTIFICIAL, opt.seed, opt.net_root_path, opt.noise_pct, MODEL_NAME)
        model, _ = load_checkpoint(model_path, model)

        ########### Do Test
        model.eval()
        with torch.no_grad():
            # total_test_metric = validate(model, test_loader, opt.N_gaussians, opt.MIN_LOSS, device,THRESHOLD=THRESHOLD)
            total_test_metric = validate_KL(model, test_loader, opt.N_gaussians, opt.MIN_LOSS, device,THRESHOLD=THRESHOLD)

        pred_metric.append(total_test_metric)

        # print(f"running time = {i}, performance = {total_test_metric}")
    min_metric = min(pred_metric)
    print(f"========== MODEL_NAME = {MODEL_NAME} ==========")
    print(f"========== best performance: {min_metric} ==========")

    metric_pd.loc[opt.seed,MODEL_NAME] = min_metric

========== MODEL_NAME = GT2(MDN) | INPUT_LIST = [2] ==========
========== opt.seed = 62 ==========
SKIP self.metric_path


IndexError: tuple index out of range

In [165]:
metric_pd

,GT2(MDN),GT1+EMD,GT2+EMD,GT1+GT2,GT1_GT2_EMD
407,3.358407,2.912842,2.993997,2.958253,2.854985


# QQQ

In [68]:
running_times = 3
metric_pd = pd.DataFrame()

for MODEL_NAME in MODEL_LIST:
    INPUT_LIST = get_Input_List(MODEL_NAME)

    print(f"========== MODEL_NAME = {MODEL_NAME} | INPUT_LIST = {INPUT_LIST} ==========")
    opt = DefaultConfig(MODEL_NAME=MODEL_NAME)

    q=opt.q
    lr_for_pi = opt.lr_for_pi
    lr_for_shape = opt.lr_for_shape
    lr_for_scale = opt.lr_for_scale
    learning_rate = opt.learning_rate

    MB_MDN_Flag = opt.Conv_1_4

    batch_size = opt.batch_size
    EPOCH_NUM = opt.EPOCH_NUM

    dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.data_key_path, opt.NLL_metric_path)

    my_collate_fn = functools.partial(my_collate_fn_3, INPUT_LIST=INPUT_LIST)

    for seed in seed_list:
        setup_seed(seed)

        print(f"========== seed = {seed} ==========")

        shuffled_indices = save_data_idx(dataset, opt.arr_flag)
        train_idx, val_idx, test_idx = get_data_idx(shuffled_indices,opt.train_pct, opt.vali_pct)

        _,_,test_loader = get_data_loader(dataset, opt.batch_size, train_idx, val_idx, test_idx, my_collate_fn)

        ans = []
        pred_metric = []

        for i in range(running_times):
            model = Conv_1_4(opt.N_gaussians, ch_in=len(INPUT_LIST)).to(device)
            model_path = get_MDN_save_path(opt.ARTIFICIAL, seed, opt.net_root_path, opt.noise_pct, MODEL_NAME)
            model, _ = load_checkpoint(model_path, model)

            ########### Do Test
            model.eval()
            with torch.no_grad():
                # total_test_metric, GT_metric,_ = validate(model, test_loader, opt.N_gaussians, opt.MIN_LOSS, device,THRESHOLD=THRESHOLD)
                total_test_metric, GT_metric = validate_KL(model, test_loader, opt.N_gaussians, opt.MIN_LOSS, device,THRESHOLD=THRESHOLD)

                GT_metric_1 = GT_metric[0, 0]

            pred_metric.append(total_test_metric)

            # print(f"running time = {i}, performance = {total_test_metric}")
        min_metric = min(pred_metric)
        print(f"========== MODEL_NAME = {MODEL_NAME} ==========")
        print(f"========== best performance: {min_metric} ==========")

        metric_pd.loc[seed,MODEL_NAME] = min_metric

========== MODEL_NAME = GT1(MDN) | INPUT_LIST = [1] ==========
========== seed = 3 ==========
THRESHOLD = 0, Number of auction config is 255
THRESHOLD = 0, Number of auction config is 255
THRESHOLD = 0, Number of auction config is 255
========== MODEL_NAME = GT1(MDN) ==========
========== best performance: 2.774993776807598 ==========
========== seed = 31 ==========
THRESHOLD = 0, Number of auction config is 255
THRESHOLD = 0, Number of auction config is 255
THRESHOLD = 0, Number of auction config is 255
========== MODEL_NAME = GT1(MDN) ==========
========== best performance: 2.846688064874387 ==========
========== seed = 204 ==========
THRESHOLD = 0, Number of auction config is 255
THRESHOLD = 0, Number of auction config is 255
THRESHOLD = 0, Number of auction config is 255
========== MODEL_NAME = GT1(MDN) ==========
========== best performance: 2.973800359987745 ==========
========== seed = 223 ==========
THRESHOLD = 0, Number of auction config is 255
THRESHOLD = 0, Number of auction

IndexError: index 3 is out of bounds for axis 0 with size 3

In [69]:
metric_pd

,GT1(MDN),GT2(MDN)
3,2.774994,3.274372
31,2.846688,3.426994
204,2.973800,3.270150
223,2.897907,3.915871
407,2.935754,3.584108


In [23]:
metric_pd

,GT1(MDN),GT2(MDN),EMD,GT1+EMD,GT2+EMD,GT1+GT2,GT1_GT2_EMD
3,6.646281,7.098381,6.903898,6.709888,6.761362,6.724608,6.624697
31,6.554226,7.193280,6.991356,6.488690,6.648332,6.569835,6.497906
204,6.593912,6.798060,6.678304,6.395668,6.523625,6.510044,6.431047
223,6.194062,7.117930,6.546107,6.163577,6.327273,6.207542,6.187846
407,6.557333,7.211479,6.932122,6.619822,6.624739,6.665380,6.510350


In [25]:
metric_pd.to_csv("output.csv",index=True,header=True)

## 3.4 Test: data set with ‘enough’ samples

In [13]:
THRESHOLD = 80

- 调查test set的比例 ()

In [14]:
for seed in [3,31,204,223,407]:
    print(f"========== seed = {seed} ==========")
    setup_seed(seed)
    dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.data_key_path, opt.NLL_metric_path)

    shuffled_indices = save_data_idx(dataset, opt.arr_flag)
    train_idx, val_idx, test_idx = get_data_idx(shuffled_indices,opt.train_pct, opt.vali_pct,opt.SET_VAL)

    my_collate_fn = functools.partial(my_collate_fn_3, INPUT_LIST=INPUT_LIST)
    _,_,test_loader = get_data_loader(dataset, opt.batch_size, train_idx, val_idx, test_idx, my_collate_fn)

    n_config_total = 0
    n_config_ge_threshold = 0
    config_ge_threshold = []
    for batch_id, data in enumerate(test_loader):
        n_config_total += 1
        input_data, target, _, setting, metric = data
        input_data = input_data.to(device)
        n_samples = target.shape[1]
        if n_samples >= THRESHOLD:
            n_config_ge_threshold += 1

    print(n_config_total,n_config_ge_threshold, n_config_ge_threshold/n_config_total)

========== seed = 3 ==========
[ 801  265  535  919  451  990  914  404 1175  490 1138  541  693  739
  364  629  459  104  833  268  608 1181 1244  904 1109  661  241  430
  814 1047  203 1234  595  630   96  467  975  492  166 1018  701  694
    9  604  878  315  773  361  138  823  276  384  568 1134  772  922
   88   90  765  280  882  651 1255  460  992  799  300 1049  202  402
  340  365   23  345  676 1235  334  310 1213  640  353  735 1246  240
  666  318 1093  262  940   98  167  496  725  275   89  714  515 1063
  331  219  433 1222 1121  486  658 1020  531  840 1099  260  956  810
  305  876 1067  702  771 1204  196  140  594  122  494  821  110   84
 1095  513  691  482 1194  369  793  926  273  396 1271  797  961  418
  375  463  470  559  302  582  798   11 1068  538  627  281  983 1264
  223  672  966 1263  994  580 1045   77  547 1272  748  826  125  757
  576  621  767 1269 1030 1152  802  844 1074 1254 1094  259  282  634
  474  455  320  434 1230  299  436 1162  438 

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.